In [29]:
import pandas as pd
import numpy as np

### Clean the UNHCR dataset
To clean the UNHCR data, I:
- Drop duplicate rows for Antigua and Barbuda
- Replace non contiguous values with contiguous values where necessary
- Remove problematic observations based on SuddenREFROCDecreases.xlsx - provided by Geraldine.

In [30]:
# remove duplicate rows
df = pd.read_csv('raw/data.csv',engine='pyarrow').drop_duplicates(['Id','year']).drop('', axis=1)

# replace contig values that should be ones but are zeros
df.loc[df.Id.isin(['CODUGA','UGACOD','SDNSSD','SSDSDN','SSDETH','ETHSSD','SSDUGA',
                   'UGASSD','CODCOG','COGCOD','CAFCOD','CODCAF','CODBDI','BDICOD',
                   'SSDKEN','KENSSD','SSDCOD','CODSSD','CODZMB','ZMBCOD','AUTHUN','HUNAUT',
                     'UVKALB','ALBUVK','MNEALB','ALBMNE','WBGISR','ISRWBG','JORWBG','WBGJOR',
                     'BGRSRB','SRBGBR','ROUSRB','SRBROU','HUNSRB','SRBHUN']), 'contig'] = 1

# sudan isn't contiguous with kenya after 2011
# df.loc[df.Id.isin(['KENSDN','SDNKEN']), 'contig'] = 0 

# remove problematic observations
df.loc[(df.Id == 'CODAGO') & df.year.isin([2017, 2018]), 'newarrival'] = np.nan

df.loc[df.Id == 'AFGIRN', 'newarrival']= np.nan

df.loc[df.Id == 'AFGPAK', 'newarrival'] = np.nan

df.loc[df.Id == 'ERTSDN', 'newarrival']= np.nan

df.loc[(df.Id == 'SYRTUR') & (df.year > 2018), 'newarrival']= np.nan

# any newarrivals after 2021 set to nan
df.loc[df.year > 2021, 'newarrival'] = np.nan

df.to_csv('clean/unhcr.csv', index=False)

Note there are some countries which the distances are the exact same for all pairs:

In [492]:
countries = df.iso_o.unique()

In [495]:
sus_country_distances = []
for country in countries:
    if len(df.loc[df.iso_o == country, 'dist'].unique()) == 1:
        sus_country_distances.append(country)

In [504]:
sus_country_distances

['COD', 'UVK', 'MNE', 'SRB', 'SSD', 'WBG']

### Cleaning the Migration data
This migration stock data comes from [Our World in Data](https://ourworldindata.org/migration#explore-data-on-where-people-migrate-from-and-to)
It is every five years from 1990 to 2020.

- I read the data in
- Add 2021
- Melt it to long format
- I forward fill the data because these migration stocks are only released every five years.
- Use country converter to convert names to iso 3 codes.

In [410]:
df = pd.read_csv('raw/migration-flows.csv', engine='pyarrow').rename({'Year':'year'},axis=1)
df = df.loc[:,~df.columns.str.contains('_origin|_destination')]
df = df[df.year >= 2000]

df['year'] = pd.to_datetime(df.year, format='%Y')

df_2021 = df[df.year == pd.to_datetime('2020', format='%Y')].copy()

# change 2020 to 2021
df_2021['year'] += pd.Timedelta(366, unit='d')

# concatenate 2021 w/ rest
df = pd.concat([df, df_2021], axis=0)

I melt the data frame, and fill in missing years.

In [411]:
# melt df, split Emigrants Immigrants columns, filter out rows with same origin and destination
df = df.melt(id_vars=['Country','year'], var_name='Country_d').rename({'Country':'Country_o'}, axis=1).fillna(0)
df[['EmigrantsImmigrants', 'Country_d']] =  df.Country_d.str.split(' from | to ', expand=True)
df = df[(df.Country_d != df.Country_o)]

# for values that say emigrants, switch the origin with destination
emigrants_mask = df['EmigrantsImmigrants'] == 'Emigrants'
df.loc[emigrants_mask, ['Country_o', 'Country_d']] = df.loc[emigrants_mask, ['Country_d', 'Country_o']].values

df = df[['year', 'Country_o', 'EmigrantsImmigrants', 'Country_d', 'value']].drop('EmigrantsImmigrants', axis=1).drop_duplicates(['year','Country_o','Country_d'])
df.value = df.value.abs()

Country long names get replaced with iso 3 short names using country converter.

In [412]:
# convert the country names to iso3
import country_converter as coco
iso3_unique = coco.convert(names=df.Country_o.unique(), to='ISO3')
iso_3_dict = dict(zip(df.Country_o.unique(), iso3_unique))

df[['Country_o','Country_d']] = df[['Country_o','Country_d']].apply(lambda x: x.map(iso_3_dict))
df.rename({'Country_o':'iso_o','Country_d':'iso_d', 'value':'migration_stock'}, axis=1,inplace=True)

Caribbean not found in regex
Channel Islands not found in regex
Polynesia not found in regex


In [413]:
# I convert stock data to the percent of migrants both from the perspective of the origin country and destination country.
unhcr_data = pd.read_csv('clean/unhcr.csv', engine='pyarrow')[['iso_o','iso_d','year','pop_o','pop_d']].drop_duplicates(['iso_o','iso_d','year'])

In [414]:
df.rename({'year':'Year'}, inplace=True, axis=1)
df['year'] = df.Year.dt.year

In [415]:
df = unhcr_data.merge(df, on=['iso_o','iso_d','year'], how='left')
df[['pop_o','pop_d']] = df[['pop_o','pop_d']].apply(lambda x: x * 1000000)
df[['migration_stock_pct_o','migration_stock_pct_d']] = df[['pop_o','pop_d']].apply(lambda x: df.migration_stock*100/x)

In [416]:
df = df.drop(['pop_o','pop_d','Year'],axis=1).rename({'Year':'year'},axis=1)

In [417]:
# fill in missing years.
df = df.set_index(['year','iso_o','iso_d']).groupby(['iso_o', 'iso_d']).fillna(method='ffill')
df.reset_index(inplace=True)
df['years_since_migration_report'] = df.year % 5

# The remaning two countries ('WBG' and 'UVK'), do mean imputation.
df[['migration_stock', 'migration_stock_pct_o','migration_stock_pct_d']] = df[['migration_stock', 'migration_stock_pct_o','migration_stock_pct_d']].apply(lambda x: x.fillna(value = x.mean()))

In [418]:
# write to csv
df.to_csv('clean/migration_stocks.csv', index=False)

### Cleaning IDP data 

This IDP data on internally displaced people comes from the IDMC https://www.internal-displacement.org/database/displacement-data.

It is yearly from 2008 to 2022, and contains data on both the stock of internally displace peoples and the new amount that year.

As a prior, let's say that disaster idp is irrelevant to refugee newarrivals and drop disaster idp data.

In [527]:
# read in an rename columns
idp_df = pd.read_excel('raw/IDMC_Internal_Displacement_Conflict-Violence_Disasters.xlsx').rename({'Year':'year','Name':'Country','ISO3':'iso','Conflict Stock Displacement (Raw)':'conflict_stock_idp', 'Conflict Internal Displacements (Raw)':'conflict_idp',
       'Disaster Internal Displacements (Raw)':'disaster_idp',
       'Disaster Stock Displacement (Raw)':'disaster_stock_idp'}, axis=1).drop(['Country','Conflict Stock Displacement','Conflict Internal Displacements','Disaster Internal Displacements','Disaster Stock Displacement', 
       'disaster_stock_idp','disaster_idp'], axis=1)

unhcr_data = pd.read_csv('raw/data.csv', engine='pyarrow')[['iso_o','year','pop_o']].drop_duplicates(['iso_o','year'])
newarrival_o = pd.read_csv('raw/data.csv', engine='pyarrow').groupby(['iso_o','year']).newarrival.sum().reset_index()
unhcr_data = unhcr_data.merge(newarrival_o, on=['iso_o','year'], how='left')

In [528]:
merged_idp = unhcr_data.merge(idp_df, how='left', left_on=['iso_o','year'], right_on=['iso','year']).drop('iso',axis=1)
#merged_idp['disaster_pct'] = (merged_idp.disaster_idp/merged_idp.pop_o/1000000)

There is a significant amount of data that are missing in the idp data:

In [529]:
print('proportion missing')
merged_idp.isna().sum() / merged_idp.shape[0]

proportion missing


iso_o                 0.000000
year                  0.000000
pop_o                 0.000000
newarrival            0.000000
conflict_stock_idp    0.852041
conflict_idp          0.864694
dtype: float64

Can we assume that this data is missing because the values should be zero?

In [530]:
# number of rows where conflict data is missing AND the number of refugees is greater than 10,000 AND the year is greater than equal to 2008
merged_idp[merged_idp.conflict_idp.isna() & (merged_idp.newarrival > 1000) & (merged_idp.year >= 2008)].sort_values(by='newarrival', ascending=False)

,iso_o,year,pop_o,newarrival,conflict_stock_idp,conflict_idp
4768,VEN,2018,28.903,2508056.5,NaN,NaN
4769,VEN,2019,27.817,1524435.0,NaN,NaN
4771,VEN,2021,27.586,664573.0,NaN,NaN
4770,VEN,2020,27.951,572356.0,NaN,NaN
3933,SOM,2008,11.400,150764.0,NaN,NaN
...,...,...,...,...,...,...
2965,MOZ,2015,27.042,1005.0,NaN,NaN
3566,ROU,2016,19.761,1004.0,NaN,NaN
3011,NAM,2011,2.116,1003.0,NaN,NaN
4760,VEN,2010,28.524,1002.0,NaN,NaN


In [531]:
# number of rows where the number of refugees is greater than 10,000 AND the year is greater than equal to 2008
merged_idp[(merged_idp.newarrival > 1000) & (merged_idp.year >= 2008) ]

,iso_o,year,pop_o,newarrival,conflict_stock_idp,conflict_idp
8,AFG,2008,22.997,29698.0,NaN,NaN
9,AFG,2009,23.596,41719.0,297000.0,0.0
10,AFG,2010,24.269,38424.0,352000.0,102000.0
11,AFG,2011,25.031,45420.0,450000.0,186000.0
12,AFG,2012,25.864,60632.0,492000.0,100400.0
...,...,...,...,...,...,...
4891,ZWE,2016,14.228,9435.0,NaN,NaN
4892,ZWE,2017,14.437,4523.0,NaN,NaN
4893,ZWE,2018,14.642,1908.0,NaN,NaN
4894,ZWE,2019,14.905,3371.0,NaN,NaN


It seems that that half of the 1/2 of the outflows where the total number of outflows is greater than 1,000 are missing. This means that missing data isn't simply zero, and it actually is missing data.

We'll need to convert this to some sort of categorical variable if we want to use it.

Here is the current approach. Where we do $$log(\frac{conflict\_variable}{population \text{ (in 1000s)}} + .0001)$$

The .0001 is to ensure that an error isn't produced.

In [532]:
merged_idp['conflict_idp_pop'] = merged_idp.apply(lambda row: np.log((row['conflict_idp'] / row['pop_o']) * 1000 + 0.1), axis=1)
merged_idp['conflict_stock_idp_pop'] = merged_idp.apply(lambda row: np.log((row['conflict_stock_idp'] / row['pop_o']) * 1000 + 0.1), axis=1)

When we do this, we get a distribution where the values are mostly between -6 and 6. The most common value before was zero, which is what the value to the left of -8 is.

To convert to categorical variable, we first convert to a string.

In [520]:
merged_idp[['conflict_stock_idp_pop', 'conflict_idp_pop']] = merged_idp[['conflict_stock_idp_pop', 'conflict_idp_pop']].apply(lambda x: np.round(x).astype(str))
merged_idp.drop(['pop_o','newarrival','conflict_stock_idp','conflict_idp'], axis=1, inplace=True)

Then we do binary encoding.

In [456]:
from category_encoders.binary import BinaryEncoder
be = BinaryEncoder()
idp_binary = be.fit_transform(merged_idp[['conflict_stock_idp_pop', 'conflict_idp_pop']])
idp_binary = pd.concat([merged_idp.drop(['conflict_stock_idp_pop', 'conflict_idp_pop'], axis=1), idp_binary],axis=1)

idp_binary.sample(10)

,iso_o,year,conflict_stock_idp_pop_0,conflict_stock_idp_pop_1,conflict_stock_idp_pop_2,conflict_stock_idp_pop_3,conflict_stock_idp_pop_4,conflict_idp_pop_0,conflict_idp_pop_1,conflict_idp_pop_2,conflict_idp_pop_3,conflict_idp_pop_4
4361,TLS,2011,0,1,1,0,1,0,0,1,1,0
2346,KGZ,2021,0,1,1,1,1,0,0,1,0,0
156,ARG,2006,0,0,0,0,1,0,0,0,0,1
2397,LVA,2022,0,0,0,0,1,0,0,0,0,1
3061,NPL,2011,0,0,1,1,0,0,0,0,1,0
3341,PAN,2016,0,0,0,0,1,0,0,0,0,1
400,BLR,2000,0,0,0,0,1,0,0,0,0,1
4384,TGO,2009,0,1,1,0,0,0,0,0,1,0
4079,LCA,2004,0,0,0,0,1,0,0,0,0,1
1507,FRA,2007,0,0,0,0,1,0,0,0,0,1


The result is 4 columns for each variable, where the values ranging from -8 to 6 or so have been encoded. The missing nan has also been encoded as a unique variable.

In [74]:
idp_binary.to_csv('clean/idp.csv')

### Fragile States Index

From [here](https://fragilestatesindex.org/)

It is yearly from 2006-2022 and contains info on 179 countries. The remaining countries we impute, taking an average of neighbors' fragilitity indices.

In [478]:
unhcr_data = pd.read_csv('clean/unhcr.csv', engine='pyarrow').drop_duplicates(['iso_o','year'])[['iso_o','year','newarrival','Country_o']]
unhcr_data = unhcr_data[(unhcr_data.year > 2005) & (unhcr_data.year < 2023)]

In [15]:
import igraph as ig
unhcr = pd.read_csv('clean/unhcr.csv', engine='pyarrow').groupby(['iso_o','iso_d']).agg({'newarrival':'sum','contig':'first','Country_o':'first','Country_d':'first', 'island_o':'first','dist':'first'}).reset_index()

df_network = unhcr[unhcr.contig == 1]

graph = ig.Graph.TupleList(df_network[['Country_o','Country_d','dist']].itertuples(index=False), directed=False, edge_attrs='dist')

# add island countries 
islands = unhcr.drop_duplicates('Country_o').sort_values('Country_o').Country_o[~unhcr.groupby('Country_o')['contig'].any().values].values

for i in islands:
    v = graph.add_vertex()
    # Set the name or other properties of the added vertex if needed
    v['name'] = i

# graph.vs['name'] = coco.convert(graph.vs['name'], to='iso3')

In [480]:
# load excel files in
import os
path = 'raw/fsi 2022-2006/'
fsi_files = os.listdir(path)
fsi_files = path + pd.Series(fsi_files)
fsi_df = pd.concat(fsi_files.apply(lambda x: pd.read_excel(x)).tolist())

In [481]:
# rename columns
fsi_df = fsi_df.rename({'Year':'year', 'Total':'fsi_overall', 'C1: Security Apparatus':'fsi_security_apparatus',
       'C2: Factionalized Elites':'fsi_factionalized_elites', 'C3: Group Grievance':'fsi_group_grievance', 'E1: Economy':'fsi_economy',
       'E2: Economic Inequality':'fsi_economic_inequality', 'E3: Human Flight and Brain Drain':'fsi_human_flight_brain_drain',
       'P1: State Legitimacy':'fsi_state_legitimacy', 'P2: Public Services':'fsi_public_services', 'P3: Human Rights':'fsi_human_rights',
       'S1: Demographic Pressures':'fsi_demographic_pressures', 'S2: Refugees and IDPs':'fsi_refugees_idps',
       'X1: External Intervention':'fsi_external_intervention'}, axis=1).drop(['Change from Previous Year','Rank'],axis=1)

# Change an error causing country name. Israel and West Bank -> West Bank and Gaza (consistent with the UNHCR naming convention)
fsi_df.loc[fsi_df.Country == 'Israel and West Bank', "Country"] = 'West Bank and Gaza'

In [525]:
# convert country name to iso3
import country_converter as coco
iso3_unique = coco.convert(names=fsi_df.Country.unique(), to='ISO3')

iso_3_dict = dict(zip(fsi_df.Country.unique(), iso3_unique))

fsi_df['iso'] = fsi_df['Country'].map(iso_3_dict)

fsi_df['year'] = fsi_df['year'].apply(lambda x: x.year if isinstance(x, pd.Timestamp) else x)

In [553]:
# for those who are missing fragility indices, filter for neighbors and take average of that
fsi_full = unhcr_data.merge(fsi_df, how='left',left_on=['iso_o','year'], right_on=['iso','year'])

In [554]:
# for those who are missing fragility indices, filter for neighbors and take average of that
missing_indices = unhcr_data.set_index(['iso_o','year'])[unhcr_data.merge(fsi_df, how='left',left_on=['iso_o','year'], right_on=['iso','year']).set_index(['iso_o','year']).fsi_overall.isna()].reset_index()[['Country_o','year']]

for index, row in missing_indices.iterrows():
    bordering = graph.vs[graph.neighborhood(row['Country_o'], order=1)]['name'][1:]
    
    # take the mean of the neighboring countries
    if len(bordering) > 0:
        fsi_full.loc[(fsi_full.Country_o == row.Country_o) & (fsi_full.year == row.year), fsi_full.columns.str.contains('fsi')] = fsi_full.loc[fsi_full.Country_o.isin(bordering) & (fsi_full.year == row['year']) , fsi_full.columns.str.contains('fsi')].agg('mean').tolist()
    # if there are no bordering countries (island), take the 5 closest countries and average. If there are still na's, then take the closest 9 countries.
    elif len(unhcr[unhcr.Country_o == row.Country_o].dist.unique()) > 1:
        idx = unhcr[unhcr.Country_o == row.Country_o].dist.nsmallest(4).index
        closest_countries = unhcr.iloc[idx].Country_d
        neighboring_fsi_indices = fsi_full.loc[fsi_full.Country_o.isin(closest_countries) & (fsi_full.year == row.year), fsi_full.columns.str.contains('fsi')].agg('mean')
        if neighboring_fsi_indices.isna().sum() == 0:
            fsi_full.loc[(fsi_full.Country_o == row.Country_o) & (fsi_full.year == row.year), fsi_full.columns.str.contains('fsi')] = neighboring_fsi_indices.tolist()
        else:
            idx = unhcr[unhcr.Country_o == row.Country_o].dist.nsmallest(9).index
            closest_countries = unhcr.iloc[idx].Country_d
            neighboring_fsi_indices = fsi_full.loc[fsi_full.Country_o.isin(closest_countries) & (fsi_full.year == row.year), fsi_full.columns.str.contains('fsi')].agg('mean')
            fsi_full.loc[(fsi_full.Country_o == row.Country_o) & (fsi_full.year == row.year), fsi_full.columns.str.contains('fsi')] = neighboring_fsi_indices.tolist()

In [571]:
fsi_full.drop(['Country','Country_o','newarrival','iso'], axis=1).rename({'iso_o':'iso'},axis=1).to_csv('clean/fragility_indices.csv', index=False)

### LDA modeled Newspaper Text/Conflict Risk

In [277]:
import pandas as pd
# we read in the data, remove nas, and drop the first column for each forecast. 
ons_armedconf1 = pd.read_csv('raw/ons_armedconf12_f_12_d_50_t_700_depth_7_mins_2_minl_50pctrial_low_pop_og.csv')
ons_armedconf1 = ons_armedconf1.loc[~ons_armedconf1.isna().any(axis=1)].iloc[:, 1:ons_armedconf1.shape[1]]

ons_armedconf2 = pd.read_csv('raw/ons_armedconf12_f_12_d_50_t_700_depth_7_mins_2_minl_100pctrial_low_pop_og.csv')
ons_armedconf2 = ons_armedconf2.loc[~ons_armedconf2.isna().any(axis=1)].iloc[:, 1:ons_armedconf2.shape[1]]

ons_armedconf3 = pd.read_csv('raw/lnbest_pc_12_f_12_d_50_t_700_depth_7_mins_2_minl_75intensity_pc_og.csv')
ons_armedconf3 = ons_armedconf3.loc[~ons_armedconf3.isna().any(axis=1)].iloc[:, 1:ons_armedconf3.shape[1]]

#lda data
lda = pd.read_csv('raw/LDA output Mueller and Rauh.csv')
lda = lda[lda.year >= 2000]

In [278]:
# merge the conflict forecasts together
merged_armed_conf = ons_armedconf1.merge(ons_armedconf2, on = ['isocode','year', 'month']).merge(ons_armedconf3, on = ['isocode','year', 'month'])

In [383]:
# fill in the missing data 
unhcr_data = pd.read_csv('clean/unhcr.csv', engine='pyarrow').drop_duplicates(['iso_o','year'])[['iso_o','year','Country_o']]
unhcr_data = unhcr_data[ (unhcr_data.year < 2023)]

lda_full = unhcr_data.merge(lda, left_on=['iso_o','year'], right_on=['isocode','year'],how='left').drop('isocode',axis=1).rename({'iso_o':'isocode'},axis=1)
conflict_full = unhcr_data.merge(merged_armed_conf, left_on=['iso_o','year'], right_on=['isocode','year'],how='left').drop('isocode',axis=1).rename({'iso_o':'isocode'},axis=1)

In [384]:
def prepare_data_for_resampling(dataframe):
    # Create a new DataFrame with a complete range of dates and reindex the original DataFrame
    min_date = pd.to_datetime('2000-01-01')
    max_date = pd.to_datetime('2023-01-01')
    reindexed_df = dataframe.reindex(pd.date_range(start=min_date, end=max_date, freq='MS'))

    return reindexed_df

def resample_missing_dates(dataframe):
    dataframe = dataframe.copy()

    dataframe.loc[dataframe.month.isna(), 'month'] = 1

    # Combine 'year' and 'month' columns into a single column
    dataframe['date'] = pd.to_datetime(dataframe['year'].astype(str) + '-' + dataframe['month'].astype(int).astype(str), format='%Y-%m')
    
    # Set the 'date' column as the index
    dataframe.set_index('date', inplace=True)
    
    # Resample the dataframe to fill missing year-month values for each country
    resampled_dataframe = dataframe.groupby(['Country_o','isocode'], as_index=False).apply(prepare_data_for_resampling)
    resampled_dataframe = resampled_dataframe.droplevel(0).groupby(['Country_o','isocode']).apply(lambda x: x.resample('MS').asfreq())

    # Reset the index to get back the 'date' column
    resampled_dataframe = resampled_dataframe.drop(['Country_o','year','month','isocode'], axis=1).reset_index().rename({'level_2':'date'},axis=1)

    # Split the 'date' column back into 'year' and 'month'
    resampled_dataframe['year'] = resampled_dataframe['date'].dt.year
    resampled_dataframe['month'] = resampled_dataframe['date'].dt.month
    resampled_dataframe.drop('date', axis=1, inplace=True)
    
    return resampled_dataframe

lda_full = resample_missing_dates(lda_full)
conflict_full = resample_missing_dates(conflict_full)

In [385]:
lda_cols = ['tokens', 'obs',
       'sentiment_words', 'uncertainty', 'ste_theta0', 'ste_theta1',
       'ste_theta2', 'ste_theta3', 'ste_theta4', 'ste_theta5', 'ste_theta6',
       'ste_theta7', 'ste_theta8', 'ste_theta9', 'ste_theta10', 'ste_theta11',
       'ste_theta12', 'ste_theta13', 'ste_theta14', 'ste_theta0_stock',
       'ste_theta1_stock', 'ste_theta2_stock', 'ste_theta3_stock',
       'ste_theta4_stock', 'ste_theta5_stock', 'ste_theta6_stock',
       'ste_theta7_stock', 'ste_theta8_stock', 'ste_theta9_stock',
       'ste_theta10_stock', 'ste_theta11_stock', 'ste_theta12_stock',
       'ste_theta13_stock', 'ste_theta14_stock', 'sentiment_stock',
       'uncertainty_index', 'best']
conflict_cols = ['ons_armedconf12_pred_jut_x',
       'ons_armedconf12_pred_top_x', 'ons_armedconf12_pred_aug_x',
       'ons_armedconf12_pred_jut_y', 'ons_armedconf12_pred_top_y',
       'ons_armedconf12_pred_aug_y', 'lnbest_pc_12_pred_jut',
       'lnbest_pc_12_pred_top', 'lnbest_pc_12_pred_aug']

In [386]:
# Helper function to calculate mean of neighboring countries
def calculate_mean_neighbors(dataframe, row, bordering, cols):
    if len(bordering) > 0:
        return dataframe.loc[dataframe.Country_o.isin(bordering) & (dataframe.year == row['year']) & (dataframe.month == row['month']), cols].agg('mean').tolist()
    else:
        idx = unhcr[unhcr.Country_o == row.Country_o].dist.nsmallest(4).index
        closest_countries = unhcr.iloc[idx].Country_d
        neighboring_indices = dataframe.loc[dataframe.Country_o.isin(closest_countries) & (dataframe.year == row.year) & (dataframe.month == row.month), cols].agg('mean')
        if neighboring_indices.isna().sum() == 0:
            return neighboring_indices.tolist()
        else:
            idx = unhcr[unhcr.Country_o == row.Country_o].dist.nsmallest(9).index
            closest_countries = unhcr.iloc[idx].Country_d
            neighboring_indices = dataframe.loc[dataframe.Country_o.isin(closest_countries) & (dataframe.year == row.year) & (dataframe.month == row.month), cols].agg('mean')
            return neighboring_indices.tolist()

def is_match(df, row):
    return (df.Country_o == row.Country_o) & (df.year == row.year) & (df.month == row.month)

# Process missing fragility indices in lda_full dataframe
missing_indices_lda = lda_full[lda_full.ste_theta0.isna()][['Country_o','year','month']]


for index, row in missing_indices_lda.iterrows():
    bordering = graph.vs[graph.neighborhood(row['Country_o'], order=1)]['name'][1:]
    lda_full.loc[is_match(lda_full, row), lda_cols] = calculate_mean_neighbors(lda_full, row, bordering, lda_cols)

# Process missing fragility indices in conflict_full dataframe
missing_indices_conflict = conflict_full[conflict_full.ons_armedconf12_pred_top_y.isna()][['Country_o','year','month']]

for index, row in missing_indices_conflict.iterrows():
    bordering = graph.vs[graph.neighborhood(row['Country_o'], order=1)]['name'][1:]
    conflict_full.loc[is_match(conflict_full, row), conflict_cols] = calculate_mean_neighbors(conflict_full, row, bordering, conflict_cols)

In [391]:
# write to file
conflict_full.to_csv('clean/mueller_conflict_forecasts.csv')
lda_full.to_csv('clean/lda.csv')

## GDELT

In [433]:
import pandas as pd
gdelt = pd.read_csv('raw/final_gdelt_bycountry.txt', engine='pyarrow')

# replace XKX code with  UNHCR code for Kosovo
gdelt.loc[gdelt.isocode == 'XKX', 'isocode'] = 'UVK'

# add WBG (just assume the same freqeuency of events as Israel)
WBG = gdelt.reset_index().copy()
WBG = WBG[WBG.isocode.isin(['ISR'])].copy().reset_index(drop=True)
WBG.loc[:, 'isocode'] = 'WBG'

gdelt = pd.concat([gdelt, WBG], axis=0)

# aggregate goldstein, taking avg goldstein score by month
goldstein = gdelt[gdelt.year >=2000].set_index(['year','month','isocode'])['AvgGoldsteinScale'].groupby(['year','month','isocode']).agg(lambda x: x.fillna(x.mean()).mean())

# if no goldstein that month, fill by year
goldstein = goldstein.groupby(['year','isocode'], group_keys=False).apply(lambda x: x.fillna(x.mean()))
# if none in the year, impute the mean
goldstein = goldstein.fillna(goldstein.mean())

gdelt = gdelt.loc[gdelt.year >=2000, ~gdelt.columns.str.contains('ADM1Code|Actor|gov|opp|Goldstein')].groupby(['year','month','isocode']).agg(sum)

# to account for the variance in recording of events across time, divide by the total number of events for that country/year
gdelt = gdelt.div(gdelt.sum(axis=1), axis=0)
gdelt = pd.concat([gdelt, goldstein], axis=1)

gdelt.to_csv('clean/gdelt.csv')